# Python tricks! : Patterns for cleaner Python

## Assertions

At its core, Python’s assert statement is a debugging aid that tests a condition. If the assert condition is true, nothing happens, and your program continues to execute as normal. But if the condition evalu- ates to false, an AssertionError exception is raised with an optional error message.

### Example:

Here’s a simple example so you can see where assertions might come in handy. I tried to give this some semblance of a real-world problem you might actually encounter in one of your programs.
Suppose you were building an online store with Python. You’re work- ing to add a discount coupon functionality to the system, and eventu- ally you write the following apply_discount function:

In [1]:
def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount)) 
    assert 0 <= price <= product['price']
    return price

Notice the assert statement in there? It will guarantee that, no mat- ter what, discounted prices calculated by this function cannot be lower

In [2]:
shoes = {'name': 'Fancy Shoes', 'price': 14900}

In [3]:
apply_discount(shoes, 0.25)

11175

In [4]:
apply_discount(shoes, 2.0)

AssertionError: 

You see, the proper use of assertions is to inform developers about unrecoverable errors in a program. Assertions are not intended to signal expected error conditions, like a File-Not-Found error, where a user can take corrective actions or just try again.

**Assertions are meant to be internal self-checks for your program. They work by declaring some conditions as impossible in your code. If one of these conditions doesn’t hold, that means there’s a bug in the pro- gram.**

If your program is bug-free, these conditions will never occur. But if they do occur, the program will crash with an assertion error telling you exactly which “impossible” condition was triggered. This makes it much easier to track down and fix bugs in your programs. And I like anything that makes life easier—don’t you?

**For now, keep in mind that Python’s assert statement is a debugging aid, not a mechanism for handling run-time errors.**

### Don’t Use Asserts for Data Validation

The biggest caveat with using asserts in Python is that assertions can be globally disabled with the -O and -OO command line switches, as well as the PYTHONOPTIMIZE environment variable in CPython.

This is an intentional design decision used similarly by many other programming languages. As a side-effect, it becomes extremely dan- gerous to use assert statements as a quick and easy way to validate input data.

In [ ]:
def delete_product(prod_id, user):
    assert user.is_admin(), 'Must be admin'
    assert store.has_product(prod_id), 'Unknown product' 
    store.get_product(prod_id).delete()

**Checking for admin privileges with an assert state- ment is dangerous.** 

If assertions are disabled in the Python interpreter, this turns into a null-op. Therefore any user can now delete products. The privileges check doesn’t even run. This likely introduces a security problem and opens the door for attackers to destroy or severely damage the data in our online store. Not good.

**The has_product() check is skipped when assertions are disabled.**

This means get_product() can now be called with invalid product IDs—which could lead to more severe bugs, depending on how our program is written. In the worst case, this could be an avenue for someone to launch Denial of Service attacks against our store. For example, if the store app crashes if someone attempts to delete an unknown product, an attacker could bombard it with invalid delete requests and cause an outage.

How might we avoid these problems? The answer is to never use as- sertions to do data validation. Instead, we could do our validation with regular if-statements and raise validation exceptions if neces- sary, like so:

In [ ]:
def delete_product(product_id, user): 
    if not user.is_admin():
        raise AuthError('Must be admin to delete') 
    if not store.has_product(product_id):
        raise ValueError('Unknown product id') 
    store.get_product(product_id).delete()

### Asserts That Never Fail

It’s surprisingly easy to accidentally write Python assert statements that always evaluate to true.

Here’s the problem, in a nutshell:
    
**When you pass a tuple as the first argument in an assert statement, the assertion always evaluates as true and therefore never fails.**

In [1]:
assert(1 == 2, 'This should fail')

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-1-2c057bd7fe24>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(1 == 2, 'This should fail')


This has to do with non-empty tuples always being truthy in Python. 

**If you pass a tuple to an assert statement, it leads to the assert condition always being true—which in turn leads to the above assert statement being useless because it can never fail and trigger an exception.**

It’s relatively easy to accidentally write bad multi-line asserts due to this, well, unintuitive behavior. 

Imagine you had this assertion in one of your unit tests:

In [3]:
counter = 10

assert (
counter == 10,
    'It should have counted all the items'
)

<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-3-737bf8fe4546>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


Upon first inspection, this test case looks completely fine. 

However, it would never catch an incorrect result: 

**The assertion always evaluates to True, regardless of the state of the counter variable.** 

And why is that? 

**Because it asserts the truth value of a tuple object.**


## Key Takeaways

* Python’s assert statement is a debugging aid that tests a condition as an internal self-check in your program.
* Asserts should only be used to help developers identify bugs. They’re not a mechanism for handling run-time errors.
* Asserts can be globally disabled with an interpreter setting.